## 1. 데이터베이스 실패
데이터베이스에서 데이터를 저장하는 것만큼 중요한 것은 저장된 데이터를 유지하는 것이다. 데이터베이스는 데이터베이스의 연산 (쿼리문)을 실행하는 동안 하드웨어, 소프트웨어, 외부의 물리적 충격 등으로 인해 시스템의 비정상적인 종료가 발생할 수 있다. 따라서, 이러한 비정상적인 종료에 대비하여 데이터베이스를 복구하기 위한 전략을 구현하는 것이 필요하다.

## 2. Transaction 단위의 데이터베이스 복구 전략
데이터베이스를 복구하기 위한 전략 중 하나는 transaction을 이용하는 것이다. 데이터베이스에서 transaction은 다수의 쿼리문으로 이루어진 논리적 작업 단위를 말한다. Transaction은 논리적 작업 단위를 구성하는 각 쿼리문의 실행마다 쿼리문의 내용을 데이터베이스에 적용하는 것이 아니라, 논리적 작업 단위가 모두 성공적으로 실행되면 COMMIT을 호출하여 모든 쿼리문을 한 번에 데이터베이스에 적용한다. Transaction의 동작을 프로그래밍 언어의 형식으로 나타내면 아래의 [그림 1]과 같다.

<img src="./image/데이터베이스복구_1.png" width="50%">

위의 [그림 1]의 11행에 있는 ROLLBACK은 transaction을 실행하는 동안 수행된 모든 쿼리문의 동작을 무효로 하는 것이다. 위의 [그림 1]에서는 단순히 ROLLBACK이라고 표현하였지만, 실제로 ROLLBACK 기능을 구현하기 위해서는 아래와 같은 사항이 필요하다.

- 데이터베이스에 연산이 적용될 때마다 변경 사항을 로그에 저장하고, 로그를 하드 디스크에 저장한다. 이를 Write-Ahead Log Protocol이라고 하며, Write-Ahead Log Protocol을 이용하는 시스템의 transaction에서는 COMMIT을 하기 전에 COMMIT 할 내용을 로그에 기록한다.

- 업데이트되는 필드, 레코드 등의 업데이트 전 값과 업데이트 후의 값을 기록한다.

## 3. Transaction과 ACID
ACID는 데이터베이스에서 transaction이 안전하게 수행된다는 것을 보장하기 위한 네 가지 성질을 말한다. ACID가 의미하는 네 가지 성질은 다음과 같다.

- 원자성 (Atomicity): transaction은 반드시 실행에 성공하거나 실패하거나 둘 중 하나의 결과만 가져야 한다. 예를 들어, 계좌 이체라는 작업을 성공하거나 실패할 수는 있지만, 출금 계좌에서 돈을 인출하는 것은 성공하고 입금 계좌에 돈을 입금하는 것만 실패하는 경우는 발생하지 말아야 한다.
- 일관성 (Consistency): transaction의 실행이 성공적으로 완료되면, 데이터베이스는 언제나 일관성 있는 상태로 유지되어야 한다. 예를 들어, 데이터베이스에 저장되는 데이터들에 적용되는 규칙을 위반하는 transaction은 실행이 중단되어야 한다.
- 고립성 (Isolation): 한 transaction을 구성하는 쿼리문들이 실행되는 중간에 다른 transaction의 쿼리문이 끼어들지 못 하게 하는 특성을 말한다.
- 지속성 (Durability): 모든 쿼리문이 성공적으로 실행되어 COMMIT 된 transaction의 내용은 영구적으로 데이터베이스에 기록되어야 한다.

ACID 속성은 transaction이 안전하게 수행된다는 것을 보장하지만, 이것을 실제로 구현하기 위해서는 인덱스 업데이트, 로그 유지, 락 (lock) 등 매우 복잡한 작업들을 추가적으로 수행해야 한다.

## 4. 데이터베이스 전체에 대한 복구 전략
위에서는 transaction을 하나의 작업 단위로 정의하여 transaction 단위의 복구 전략을 설명하였다. 따라서, 데이터베이스 전체가 비정상적으로 종료되어도 transaction 단위로 복구를 한다면, 데이터베이스를 완벽히 복구할 수 있다고 생각할 수 있다. 그러나, transaction이 COMMIT 되어 데이터베이스에 변경 사항을 기록하는 동안 데이터베이스가 비정상적으로 종료되었을 경우에는 transaction은 COMMIT이 되어 복구의 대상이 되지 않지만, transaction의 변경 내용은 데이터베이스에 적용되지 않는 문제가 발생한다.

데이터베이스에서는 이러한 문제를 해결하기 위해 일정 주기마다 checkpoint를 설정한다. 아래의 [그림 2]는 checkpoint를 이용한 데이터베이스 복구 전략을 설명하기 위한 예시이다. 아래의 [그림 2]에서 T1, T2, ..., T5는 transaction을, Tc는 checkpoint를 의미한다

<img src="./image/데이터베이스복구_2.png" width="50%">

Checkpoint에서는 현재 실행되고 있는 transaction의 내용을 로그에 기록한다. 따라서, [그림 2]와 같은 상황에서는 T2와 T3의 내용이 로그에 기록된다.

위의 [그림 2]에서는 하나의 checkpoint 이후 system failure가 발생하였다. 이와 같은 system failure 발생 이후에 데이터베이스는 각각의 transaction에 대해 다음과 같은 작업을 수행한다.

- T1: 아무런 작업도 수행하지 않는다.

- T2와 T4: COMMIT이 완료되었기 때문에 로그에서 해당 transaction의 내용을 읽어 데이터베이스에 적용한다. 이러한 동작을 redo라고 한다.

- T3와 T5: COMMIT이 완료되지 않은 상태로 시스템이 비정상적으로 종료되었기 때문에 로그를 읽어 해당 transaction의 동작을 모두 취소하고, 처음부터 다시 실행한다. 이러한 동작을 undo라고 한다.